In [11]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding
import random
import pandas as pd

# Step 1: Load a blank SpaCy model
nlp = spacy.blank("en")

# Step 2: Add the text classification pipeline component to the model
if "textcat" not in nlp.pipe_names:
    textcat = nlp.add_pipe("textcat", last=True)
else:
    textcat = nlp.get_pipe("textcat")

# Step 3: Add labels for sentiment classification (positive, neutral, negative)
textcat.add_label("POSITIVE")
textcat.add_label("NEUTRAL")
textcat.add_label("NEGATIVE")

# Define a function to map star ratings to sentiment categories
def map_rating_to_label(rating):
    if rating >= 4:  # 4-5 stars as POSITIVE
        return {"POSITIVE": 1, "NEUTRAL": 0, "NEGATIVE": 0}
    elif rating == 3:  # 3 stars as NEUTRAL
        return {"POSITIVE": 0, "NEUTRAL": 1, "NEGATIVE": 0}
    else:  # 1-2 stars as NEGATIVE
        return {"POSITIVE": 0, "NEUTRAL": 0, "NEGATIVE": 1}

# Step 4: Load the list of file paths
# Assuming you have a file `file_paths.txt` that contains paths to CSV files, one per line
with open('csv_Files', 'r') as file:
    file_paths = [line.strip() for line in file]

# Initialize an empty list for storing training data
train_data = []

# Loop through each file path and process the data
for file_path in file_paths:
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Clean and process the data
    df = df.dropna(subset=['Review Text', 'Rating'])  # Drop rows with missing review text or rating
    df['Review Text'] = df['Review Text'].astype(str)  # Ensure all review texts are strings

    # Iterate through each row in the DataFrame and process the review text and rating
    for index, row in df.iterrows():
        text = row['Review Text']
        rating = row['Rating']
        cats = map_rating_to_label(rating)
        train_data.append((text, {"cats": cats}))

# Step 5: Train the model with the combined data
# Disable other pipeline components to only train the textcat component
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]
with nlp.disable_pipes(*other_pipes):  # Only train textcat
    optimizer = nlp.begin_training()
    for epoch in range(10):  # Train for 10 epochs
        random.shuffle(train_data)
        losses = {}
        # Use minibatch training
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            examples = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                examples.append(Example.from_dict(doc, annotations))
            nlp.update(examples, drop=0.2, losses=losses)
        print(f"Epoch {epoch + 1} Loss: {losses['textcat']}")

# Step 6: Test the model with new examples
test_texts = [
    "I absolutely love this!",
    "This is terrible and I hate it.",
    "I am not sure how I feel about this.",
]

for text in test_texts:
    doc = nlp(text)
    print(text, doc.cats)  # Returns a dictionary with POSITIVE/NEUTRAL/NEGATIVE scores

# Step 7: Save the trained model to a directory
nlp.to_disk("sentiment_model")

# To load the model later for use
# nlp2 = spacy.load("sentiment_model")

/home/renju/CP/mouthShut/Bajaj_Pulsar_180F/Bajaj_Pulsar_180F_page_1.csv
/home/renju/CP/mouthShut/Bajaj_Platina_99/Bajaj_Platina_99_page_1.csv
/home/renju/CP/mouthShut/Bajaj_XCD_Sprint/Bajaj_XCD_Sprint_page_1.csv
/home/renju/CP/mouthShut/Bajaj_Byk_M/Bajaj_Byk_M_page_1.csv
/home/renju/CP/mouthShut/Bajaj_Caliber_Croma/Bajaj_Caliber_Croma_page_1.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_37.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_25.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_13.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_6.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_29.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_44.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_14.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pulsar_NS160_page_10.csv
/home/renju/CP/mouthShut/Bajaj_Pulsar_NS160/Bajaj_Pul

TypeError: object of type 'float' has no len()